In [ ]:
import geopandas as gpd
import pandas as pd
import cartopy
import matplotlib.pyplot as plt
import json
import shapely

import shared_functions as sf

# settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 150)

In [ ]:
with open('metadata/SPOT_L1A.json') as f:
    data = json.load(f)
df=pd.json_normalize([data[k] for k in data.keys()])
df['time'] = pd.to_datetime(df['content.properties.DataDate'])
df['month'] = df['time'].dt.month
df['month_name'] = df['time'].dt.month_name()
df['year'] = df['time'].dt.year
df["geometry"] = df["content.properties.PolygonWKT"].apply(lambda x : shapely.wkt.loads(x))
df = gpd.GeoDataFrame(df, geometry="geometry", crs = "EPSG:4326")
df = df.to_crs(3031)
df = sf.filter_results_with_geojson(df, filename="shapefiles/50south_excl_argentina_falkand_mid.geojson")

In [ ]:
df.geometry.bounds.miny

In [ ]:
title = 'Spot Satellite Coverage by Generation (1984-2015)'
sf.plot_results_footprint_map(df, title=title, group='content.properties.PlatformName', legend_title='Satellite')

## Summarise

In [ ]:
sf.plot_timeseries_products(df, 
	title=f'Spot (Level 1A) - Weekly Products' ,
	stack_col='content.properties.PlatformName', 
	date_col='time',
	count_freq='7D', 
    plot_freq='3M')

In [ ]:
#mean filesize
df.groupby('content.properties.PlatformName')['content.properties.DataFileSize'].mean()/1E6

### Product Count and Volume

In [ ]:
list(df)
count_size = (
    (df.groupby(['year','content.properties.PlatformName',]).agg(
    count = ('content.id','count'),
    size = ('content.properties.DataFileSize', 'sum')))
    .reset_index()
    .pivot(index='year',columns='content.properties.PlatformName', values=['count','size'])
    .fillna(0)
)
count_size.loc["Total"] = count_size.sum()
count_size['count'] = count_size['count'].astype(int)
count_size['size'] = (count_size['size'].astype(int)/1E9).round(2)
count_size

In [ ]:
time = df.groupby(['content.properties.PlatformName',]).agg(
    min_time = ('time','min'),
    max_time = ('time', 'max')).reset_index()
time['min_time'] = time['min_time'].dt.date
time['max_time'] = time['max_time'].dt.date
time

In [ ]:
size = df.groupby(['content.properties.PlatformName',]).agg(
    size = ('content.properties.DataFileSize','mean')).reset_index()

size['size'] = (size['size'].astype(int)/1E6).round(0)
size

In [ ]:
sf.plot_frequency(df[df['year']==2008],
                  title = 'All Spot Observations (2018)',
                  cloud_col= 'content.properties.CloudCover',
                  shape=(1000,1000),
                  plot_cloud=True,
                  #force_max=30
                )

## SPOT 5 by year

In [ ]:
sf.plot_multiple_frequency(
    df[df['content.properties.PlatformName'].isin(['SPOT5'])],
      title = 'SPOT-5 revisit frequency by year',
      group='year', 
      sort_group='year',
      cloud_col= 'content.properties.CloudCover',
      shape=(1500,1500),
      plot_cloud=False,
      n_cols=5,
      force_max=50
    )

In [ ]:
sf.plot_multiple_frequency(
    df[df['content.properties.PlatformName'].isin(['SPOT5'])],
      title = 'SPOT-5 revisit frequency by year',
      group='year', 
      sort_group='year',
      cloud_col= 'content.properties.CloudCover',
      shape=(1000,1000),
      plot_cloud=True,
      n_cols=4
      #force_max=50
    )

## Plot All by Satellite

In [ ]:
sf.plot_multiple_frequency(
    df,
    title = 'All Spot Observations by Generation',
    group='content.properties.PlatformName', 
    sort_group='year',
    cloud_col= 'content.properties.CloudCover',
    shape=(1000,1000),
    n_cols=3,
    #plot_cloud=True,
    #force_max=50
)